# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## AFRICAN

## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [13]:
# Check directory where AMP-PD data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AFR/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/chr9_AFR_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/chr9_AFR_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/chr9_AFR_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/chr9_AFR_release3.pvar...
- [4/4 files][247.3 MiB/247.3 MiB] 100% Done                                    
Operation completed over 4 objects/247.3 MiB.                                    


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [22]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [23]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [24]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_AFR_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_AFR

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AFR.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_AFR
  --pfile chr9_AFR_release3
  --to-bp 129148946

Start time: Thu Jun  1 15:31:19 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
404 samples (250 females, 154 males; 404 founders) loaded from
chr9_AFR_release3.psam.
863361 variants loaded from chr9_AFR_release3.pvar.
1 binary phenotype loaded (49 cases, 355 controls).
305 variants remaining after main filters.
Writing pheno_PP2A_AFR.fam ... done.
Writing pheno_PP2A_AFR.bim ... done.
Writing pheno_PP2A_AFR.bed ... 0%done.
End time: Thu Jun  1 15:31:20 2023


### Visualize plink files bim, fam and bed

In [25]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AFR.bim

9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111023:G:C	0	129111023	C	G
9	chr9:129111059:G:A	0	129111059	A	G
9	chr9:129111157:A:T	0	129111157	T	A
9	chr9:129111177:G:A	0	129111177	A	G
9	chr9:129111181:T:C	0	129111181	C	T
9	chr9:129111184:G:C	0	129111184	C	G
9	chr9:129111349:C:T	0	129111349	T	C
9	chr9:129111359:T:G	0	129111359	G	T
9	chr9:129111392:G:C	0	129111392	C	G


In [26]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AFR.fam

0	BCM_000224_s1	0	0	1	2
0	BCM_000315_s1	0	0	2	2
0	BCM_000358_s1	0	0	1	2
0	BCM_000402_s1	0	0	2	1
0	BCM_000415_s1	0	0	1	2
0	BCM_000613_s1	0	0	1	1
0	BCM_000729_s1	0	0	2	1
0	BIOFIND_000001_s1	0	0	1	1
0	BIOFIND_000002_s1	0	0	1	1
0	BLAACPD-KPM_000003_s1	0	0	2	1


In [27]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_AFR.bed

l��������������������������߻�����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������/��������ߺ����{�����������������������������������;���������������������������������������������/��������ߺ����{�����������������������������������;�������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������/��������ߺ����{�����������������������������������;�������������������������������������������������������������������������������������������������������         �                                     �    @            @             @      �������������������������������������������/��������ߺ����{�����������������������������������;��������

In [28]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_AFR \
--recode vcf-fid \
--out pheno_PP2A_AFR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_AFR.log.
Options in effect:
  --bfile pheno_PP2A_AFR
  --out pheno_PP2A_AFR
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%

## Annotate PP2A variants

In [29]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A_AFR.vcf

tabix -f -p vcf pheno_PP2A_AFR.vcf.gz
tabix -f -p vcf pheno_PP2A_AFR.vcf.gz

[bgzip] can't create pheno_PP2A_AFR.vcf.gz: File exists


In [30]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AFR.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A_AFR.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A_AFR.vcf.gz > pheno_PP2A_AFR.annovar.avinput>
NOTICE: Finished reading 312 lines from VCF file
NOTICE: A total of 305 locus in VCF file passed QC threshold, representing 285 SNPs (208 transitions and 77 transversions) and 20 indels/substitutions
NOTICE: Finished writing allele frequencies based on 115140 SNP genotypes (84032 transitions and 31108 transversions) and 8080 indels/substitutions for 404 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A_AFR.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A_AFR.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno 

## Extract coding and non-syn variants

In [31]:
# Visualize multianno file
PP2A_AFR = pd.read_csv(f'{WORK_DIR}/pheno_PP2A_AFR.annovar.hg38_multianno.txt', sep = '\t')
PP2A_AFR.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo407,Otherinfo408,Otherinfo409,Otherinfo410,Otherinfo411,Otherinfo412,Otherinfo413,Otherinfo414,Otherinfo415,Otherinfo416
0,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0
1,9,129111023,129111023,G,C,UTR5,PTPA,NM_021131:c.-578G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,9,129111059,129111059,G,A,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/1,0/1,0/1,0/1,0/0,0/0
3,9,129111157,129111157,A,T,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/1,0/1,0/1,0/1,0/0,0/0
4,9,129111177,129111177,G,A,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [32]:
PP2A_AFR.count()

Chr             305
Start           305
End             305
Ref             305
Alt             305
               ... 
Otherinfo412    305
Otherinfo413    305
Otherinfo414    305
Otherinfo415    305
Otherinfo416    305
Length: 427, dtype: int64

In [33]:
# Filter exonic variants
coding_AFR = PP2A_AFR[PP2A_AFR['Func.refGene'] == 'exonic']
coding_AFR.count()
coding_AFR.head(5)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo407,Otherinfo408,Otherinfo409,Otherinfo410,Otherinfo411,Otherinfo412,Otherinfo413,Otherinfo414,Otherinfo415,Otherinfo416
11,9,129111606,129111606,T,C,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon1:c.T6C:p.A2A,PTPA:NM_17...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1
96,9,129123135,129123135,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001193397:exon3:c.C108T:p.S36S,PTPA:NM...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
184,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
199,9,129136522,129136522,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon6:c.C525T:p.A175A,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
286,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [34]:
# Filter exonic and non-syn vars
coding_nonsynonymous_AFR = PP2A_AFR[(PP2A_AFR['Func.refGene'] == 'exonic') & (PP2A_AFR['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_AFR.count(1)

184    427
286    427
dtype: int64

In [35]:
coding_nonsynonymous_AFR.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo407,Otherinfo408,Otherinfo409,Otherinfo410,Otherinfo411,Otherinfo412,Otherinfo413,Otherinfo414,Otherinfo415,Otherinfo416
184,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [36]:
print(coding_nonsynonymous_AFR)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
184    9  129134882  129134882   A   C       exonic         PTPA   
286    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
184                  .  nonsynonymous SNV   
286                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo407  \
184  PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...  ...          0/0   
286  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...          0/0   

     Otherinfo408 Otherinfo409 Otherinfo410  Otherinfo411  Otherinfo412  \
184           0/0          0/0          0/0           0/0           0/0   
286           0/0          0/0          0/0           0/0           0/0   

    Otherinfo413 Otherinfo414 Otherinfo415 Otherinfo416  
184          0/0          0/0          0/0          0/0  
286          0/0          0/0          0/0          0/0  

[2 rows x 427 columns]


In [37]:
coding_nonsynonymous_AFR.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_AFR.head(9)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo407,Otherinfo408,Otherinfo409,Otherinfo410,Otherinfo411,Otherinfo412,Otherinfo413,Otherinfo414,Otherinfo415,Otherinfo416
184,9,129134882,129134882,A,C,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon5:c.A461C:p.K154T,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
286,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [38]:
# Filter intronic variants
intronic_AFR = PP2A_AFR[PP2A_AFR['Func.refGene'] == 'intronic']
intronic_AFR.count()

Chr             276
Start           276
End             276
Ref             276
Alt             276
               ... 
Otherinfo412    276
Otherinfo413    276
Otherinfo414    276
Otherinfo415    276
Otherinfo416    276
Length: 427, dtype: int64

In [39]:
# Filter UTR3 variants
UTR3_AFR = PP2A_AFR[PP2A_AFR['Func.refGene'] == ('UTR3')]
UTR3_AFR.count()

Chr             18
Start           18
End             18
Ref             18
Alt             18
                ..
Otherinfo412    18
Otherinfo413    18
Otherinfo414    18
Otherinfo415    18
Otherinfo416    18
Length: 427, dtype: int64

In [40]:
# Filter UTR5 variants
UTR5_AFR = PP2A_AFR[PP2A_AFR['Func.refGene'] == ('UTR5')]
UTR5_AFR.count()

Chr             6
Start           6
End             6
Ref             6
Alt             6
               ..
Otherinfo412    6
Otherinfo413    6
Otherinfo414    6
Otherinfo415    6
Otherinfo416    6
Length: 427, dtype: int64

In [41]:
# Calculate freq - cases vs controls

In [42]:
reduced_coding_nonsynonymous_AFR = coding_nonsynonymous_AFR[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_AFR.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_AFR.txt', sep = '\t', index = False, header= False)

In [43]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_AFR.txt

9	129134882	129134882	PTPA
9	129147457	129147457	PTPA


In [44]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AFR --extract range reduced_coding_nonsynonymous_AFR.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AFR --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AFR.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AFR
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_AFR.txt
  --out coding_nonsynonymous_pheno_PP2A_AFR

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
--extract range: 303 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%3

In [45]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AFR --extract range reduced_coding_nonsynonymous_AFR.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_AFR_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_AFR_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_AFR
  --extract range reduced_coding_nonsynonymous_AFR.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_AFR_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
--extract range: 303 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%3

In [46]:
PP2A_AFR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AFR.assoc', delim_whitespace=True)
PP2A_AFR_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129134882:A:C,129134882,C,0.00000,0.004249,A,0.418,0.5179,0.000,inf,0.0000,NaN
1,9,chr9:129147457:C:T,129147457,T,0.02041,0.008475,C,1.248,0.2640,2.438,0.8237,0.4851,12.25


In [47]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AFR --extract range reduced_coding_nonsynonymous_AFR.txt --assoc --out coding_nonsynonymous_pheno_PP2A_AFR --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AFR.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_AFR
  --extract range reduced_coding_nonsynonymous_AFR.txt
  --out coding_nonsynonymous_pheno_PP2A_AFR

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
--extract range: 303 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33

In [48]:
PP2A_AFR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AFR.assoc.adjusted', delim_whitespace=True)
PP2A_AFR_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129147457:C:T,0.2640,0.4085,0.5281,0.5281,0.4583,0.4583,0.5179,0.7769
1,9,chr9:129134882:A:C,0.5179,0.6324,1.0000,0.5281,0.7676,0.5179,0.5179,0.7769


## Calculate freq of HMZ in cases versus controls

In [49]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_AFR --extract range reduced_coding_nonsynonymous_AFR.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AFR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AFR.log.
Options in effect:
  --bfile pheno_PP2A_AFR
  --extract range reduced_coding_nonsynonymous_AFR.txt
  --out coding_nonsynonymous_pheno_PP2A_AFR
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
--extract range: 303 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [50]:
PP2A_AFR_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AFR.assoc', delim_whitespace=True)
PP2A_AFR_freq.head()

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
0,9,chr9:129134882:A:C,129134882,C,0.00000,0.004249,A,0.418,0.5179,0.000
1,9,chr9:129147457:C:T,129147457,T,0.02041,0.008475,C,1.248,0.2640,2.438


In [51]:
PP2A_AFR_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A_AFR.raw', delim_whitespace=True)
PP2A_AFR_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,0,BCM_000224_s1,0,0,1,2,0.0,0.0
1,0,BCM_000315_s1,0,0,2,2,0.0,0.0
2,0,BCM_000358_s1,0,0,1,2,0.0,0.0
3,0,BCM_000402_s1,0,0,2,1,0.0,0.0
4,0,BCM_000415_s1,0,0,1,2,0.0,0.0


In [59]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_PP2A_AFR --extract range reduced_coding_nonsynonymous_AFR.txt --recode A --out coding_nonsynonymous_pheno_PP2A_AFR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A_AFR.log.
Options in effect:
  --bfile pheno_PP2A_AFR
  --extract range reduced_coding_nonsynonymous_AFR.txt
  --out coding_nonsynonymous_pheno_PP2A_AFR
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
305 variants loaded from .bim file.
404 people (154 males, 250 females) loaded from .fam.
404 phenotype values loaded from .fam.
--extract range: 303 variants excluded.
--extract range: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%

In [60]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos_AFR = PP2A_AFR_recode[(PP2A_AFR_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AFR_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos_AFR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T


In [61]:
PP2A_hom_casos_AFR.shape

(0, 8)

In [62]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles_AFR = PP2A_AFR_recode[(PP2A_AFR_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_AFR_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles_AFR.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T


In [63]:
PP2A_hom_controles_AFR.shape

(0, 8)

In [64]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_AFR = PP2A_AFR_recode[(PP2A_AFR_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AFR_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_AFR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
37,0,BLAACPD-UAB_000018_s1,0,0,1,2,0.0,1.0
53,0,CORIELL_000737_s1,0,0,2,2,0.0,1.0


In [66]:
# Explore HETs for chr9:129147457:C:T_T in controls
PP2A_het_controles_129147457_AFR = PP2A_AFR_recode[(PP2A_AFR_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_AFR_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_AFR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
152,0,CORIELL_007419_s1,0,0,2,1,0.0,1.0
246,0,CORIELL_007879_s1,0,0,2,1,0.0,1.0
253,0,CORIELL_007922_s1,0,0,2,1,0.0,1.0
258,0,CORIELL_007950_s1,0,0,2,1,0.0,1.0
317,0,CORIELL_008214_s1,0,0,2,1,0.0,1.0
357,0,CORIELL_008406_s1,0,0,2,1,0.0,1.0


In [67]:
!pip install rpy2
%load_ext rpy2.ipython

In [69]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [103]:
PP2A_het_casos_129147457_AFR.to_csv('PP2A_het_casos_129147457_AFR.csv')

In [104]:
temp = pd.read_csv('PP2A_het_casos_129147457_AFR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,37,0,BLAACPD-UAB_000018_s1,0,0,1,2,0.0,1.0
1,53,0,CORIELL_000737_s1,0,0,2,2,0.0,1.0


In [77]:
PP2A_het_controles_129147457_AFR.to_csv('PP2A_het_controles_129147457_AFR.csv')

In [78]:
temp = pd.read_csv('PP2A_het_controles_129147457_AFR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,152,0,CORIELL_007419_s1,0,0,2,1,0.0,1.0
1,246,0,CORIELL_007879_s1,0,0,2,1,0.0,1.0
2,253,0,CORIELL_007922_s1,0,0,2,1,0.0,1.0
3,258,0,CORIELL_007950_s1,0,0,2,1,0.0,1.0
4,317,0,CORIELL_008214_s1,0,0,2,1,0.0,1.0
5,357,0,CORIELL_008406_s1,0,0,2,1,0.0,1.0


In [102]:
PP2A_het_casos_129147457_AFR.shape

(2, 8)

In [74]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

data.table 1.14.6 using 2 threads (see ?getDTthreads).  Latest news: r-datatable.com
In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/home/jupyter/packages’, ‘/usr/lib/R/site-library’ contain no packages


In [93]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [100]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_AFR <- fread("PP2A_het_casos_129147457_AFR.csv", header =T)
PP2A_het_casos_129147457_AFR
temp <- merge(covs, PP2A_het_casos_129147457_AFR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_AFR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_AFR_extracted <- fread("PP2A_het_casos_129147457_AFR_extracted.txt", header =T)
PP2A_het_casos_129147457_AFR_extracted

Error in fread("PP2A_het_casos_129147457_AFR.csv", header = T) : 
  File 'PP2A_het_casos_129147457_AFR.csv' does not exist or is non-readable. getwd()=='/home/jupyter/PP2A_GP2'


RInterpreterError: Failed to parse and evaluate line 'setwd(\'/home/jupyter/PP2A_GP2/\')\nlibrary(data.table)\nPP2A_het_casos_129147457_AFR <- fread("PP2A_het_casos_129147457_AFR.csv", header =T)\nPP2A_het_casos_129147457_AFR\ntemp <- merge(covs, PP2A_het_casos_129147457_AFR, by="IID")\nhead(temp)\nwrite.table(temp, file = "PP2A_het_casos_129147457_AFR_extracted.txt", quote = F, row.names = F, sep = "\\t")\nPP2A_het_casos_129147457_AFR_extracted <- fread("PP2A_het_casos_129147457_AFR_extracted.txt", header =T)\nPP2A_het_casos_129147457_AFR_extracted\n'.
R error message: 'Error in fread("PP2A_het_casos_129147457_AFR.csv", header = T) : \n  File \'PP2A_het_casos_129147457_AFR.csv\' does not exist or is non-readable. getwd()==\'/home/jupyter/PP2A_GP2\''

In [97]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_AFR <- fread("PP2A_het_controles_129147457_AFR.csv", header =T)
PP2A_het_controles_129147457_AFR
temp <- merge(covs, PP2A_het_controles_129147457_AFR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129147457_AFR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_AFR_extracted <- fread("PP2A_het_controles_129147457_AFR_extracted.txt", header =T)
PP2A_het_controles_129147457_AFR_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1: CORIELL_007419_s1 CORIELL_007419          m6   Control            1
2: CORIELL_007879_s1 CORIELL_007879          m6   Control            1
3: CORIELL_007922_s1 CORIELL_007922          m6   Control            1
4: CORIELL_007950_s1 CORIELL_007950          m6   Control            1
5: CORIELL_008214_s1 CORIELL_008214          m6   Control            1
6: CORIELL_008406_s1 CORIELL_008406          m6   Control            1
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:     Control          2  74           NA               NA           NA
2:     Control          2  68           NA               NA           NA
3:     Control          2  60           NA               NA           NA
4:     Control          2  73           NA               NA           NA
5:     Control          2  64           NA               NA           NA
6:     Control          2  66           NA               NA      

In [86]:
# Explore HETs for chr9:129134882:A:C_C in controls
PP2A_het_controles_129134882_AFR = PP2A_AFR_recode[(PP2A_AFR_recode['chr9:129134882:A:C_C'] == 1) & (PP2A_AFR_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129134882_AFR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
82,0,CORIELL_005817_s1,0,0,1,1,1.0,0.0
139,0,CORIELL_007374_s1,0,0,1,1,1.0,0.0
301,0,CORIELL_008140_s1,0,0,1,1,1.0,0.0


In [87]:
PP2A_het_controles_129134882_AFR.to_csv('PP2A_het_controles_129134882_AFR.csv')

In [88]:
temp = pd.read_csv('PP2A_het_controles_129134882_AFR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129134882:A:C_C,chr9:129147457:C:T_T
0,82,0,CORIELL_005817_s1,0,0,1,1,1.0,0.0
1,139,0,CORIELL_007374_s1,0,0,1,1,1.0,0.0
2,301,0,CORIELL_008140_s1,0,0,1,1,1.0,0.0


In [89]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129134882_AFR <- fread("PP2A_het_controles_129134882_AFR.csv", header =T)
PP2A_het_controles_129134882_AFR
temp <- merge(covs, PP2A_het_controles_129134882_AFR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129134882_AFR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129134882_AFR_extracted <- fread("PP2A_het_controles_129134882_AFR_extracted.txt", header =T)
PP2A_het_controles_129134882_AFR_extracted

                 IID          GP2ID manifest_id phenotype pheno_for_qc
1: CORIELL_005817_s1 CORIELL_005817          m6   Control            1
2: CORIELL_007374_s1 CORIELL_007374          m6   Control            1
3: CORIELL_008140_s1 CORIELL_008140          m6   Control            1
   other_pheno sex_for_qc age age_of_onset age_at_diagnosis age_at_death
1:     Control          1  69           NA               NA           NA
2:     Control          1  59           NA               NA           NA
3:     Control          1  62           NA               NA           NA
                 race_for_qc family_history_for_qc region_for_qc   study pruned
1: Black or African American                    No           USA CORIELL      0
2: Black or African American          Not Reported           USA CORIELL      0
3: Black or African American                    No           USA CORIELL      0
   pruned_reason label related  V1 FID PAT MAT SEX PHENOTYPE
1:            NA   AFR      NA  82   0   0 

## Save out results..!

In [90]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PP2A_GP2/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_CAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_controles_129147457_EUR_extracted.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AJ_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AFR.fam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AAC.vcf.gz.tbi [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_EAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AAC_rare_0.05.bed [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.annovar.h